In [1]:
import os
import numpy as np

from matchedfilterGPU import matched_filter

In [2]:
data=np.load('data/noisy_data.npy')
templates=np.load('data/noisy_template.npy')
pads=np.load('data/noisy_pads.npy')

# templates=templates[:,:,:,::10]
for _ in range (0,5):
    data=np.concatenate((data,data),axis=2)

print("Trying to compute with traces format")
n_templates = templates.shape[0]
n_stations = templates.shape[1]
n_components = templates.shape[2]
n_samples_template =templates.shape[3]
n_samples_data = data.shape[-1]

weights = np.ones(
    (len(templates), len(data)))
weights /= len(data)

Trying to compute with traces format


In [3]:
data.shape

(5, 1, 3200000)

In [4]:
templates.shape

(1, 5, 1, 600)

In [5]:
n_templates

1

In [6]:
pads.shape

(1, 5)

In [7]:
pads

array([[146.,  65., 211., 131.,   0.]], dtype=float32)

In [8]:
templates = templates.reshape(
    n_templates, n_stations * n_components, n_samples_template)
weights_alt = weights.reshape(
    n_templates, n_stations * n_components)
moveouts = pads.reshape(
    n_templates, n_stations * n_components)
data = data.reshape(
    n_stations * n_components, n_samples_data)

In [9]:
import ctypes as ct

In [10]:
sum_square_templates = np.sum(templates**2, axis=-1)
sum_square_templates = np.ascontiguousarray(
    sum_square_templates.flatten(), dtype=np.float32
)


step=1

max_samples_template = np.max(n_samples_template)
n_corr = int((n_samples_data - max_samples_template) / step + 1)

cc = np.zeros(
    n_templates * n_stations * n_components * (n_corr+1), dtype=np.float32
)


In [11]:
templates = np.ascontiguousarray(templates.flatten(), dtype=np.float32)
moveouts= np.ascontiguousarray(moveouts.flatten(), dtype=np.int32)
data = np.ascontiguousarray(data.flatten(), dtype=np.float32)
weights = np.ascontiguousarray(weights.flatten(), dtype=np.float32)
step=int(step)


In [12]:
n_templates

1

In [13]:
STOP

NameError: name 'STOP' is not defined

In [14]:

matched_filter(templates, sum_square_templates, moveouts, 
               data, weights, step, n_samples_template, n_samples_data, n_templates,
               n_stations, n_components, n_corr, cc)   



    
    # void matched_filter(float *templates, float *sum_square_templates,
    #                     int *moveouts, float *data, float *weights, size_t step,
    #                     size_t n_samples_template, size_t n_samples_data,
    #                     size_t n_templates, size_t n_stations,
    #                     size_t n_components, size_t n_corr,
    #                     float *cc_out, int normalize, int sum_cc_mode)
    
    

In [15]:
cc[1]

-0.018576073

In [ ]:
                    network_corr<<<GS, BS, sharedMem>>>(templates_d_t,
                                                        sum_square_templates_d_t,
                                                        moveouts_d_t,
                                                        data_d,
                                                        weights_d_t,
                                                        step,
                                                        n_samples_template,
                                                        n_samples_data,
                                                        n_stations,
                                                        n_components,
                                                        chunk_offset,
                                                        cs,
                                                        cc_mat_d,
                                                        normalize);


In [ ]:
# import os
# import numpy as np

# from cuda_add import add

# import numpy as np
# import cuda_add  # Ensure this module name matches the name given in PYBIND11_MODULE

# # Create two NumPy arrays with random values
# x = np.random.rand(6024000000).astype('float32')
# y = np.random.rand(6024000000).astype('float32')

# # Print original arrays
# print("Original x: ", x[:5])  # Print first 5 elements to check
# print("Original y: ", y[:5])

# # Call the add function from your CUDA module
# add(x, y)  # This will modify 'y' in place

# # Print the result
# print("Modified y (x + y): ", y[:5])
